In [1]:
import pandas as pd
import numpy as np
import time
import ast
from astropy.time import Time

In [2]:
data = pd.read_csv('R3_intensity_pub.csv', delimiter=',')
print(data.columns.tolist())
print(data["Bandwidth"])

['event_id', 'fitburst_snr', 'fitburst_dm', 'fitburst_dm_error', 'num_sub_bursts', 'pulse_width_ms', 'pulse_width_ms_error', 'scattering_time_ms', 'scattering_time_ms_error', 'fitburst_amplitude', 'fitburst_amplitude_error', 'spectral_index', 'spectral_index_error', 'spectral_running', 'spectral_running_error', 'Bandwidth', 'Central Frequency', 'UTC_400', 'UTC_400_err', 'type_of_data']
0                      0.0
1                      0.0
2                      0.0
3                      0.0
4                      0.0
5                      0.0
6                      0.0
7                      0.0
8                      0.0
9      [40.37254951976685]
10     [51.90657993311264]
11    [28.610219874167107]
12    [11.508242108998923]
13    [21.872062511916205]
14     [26.19960077763819]
Name: Bandwidth, dtype: object


In [3]:
freq_bottom_mhz = 400.1953125
freq_top_mhz = 800.1953125
freq = np.arange(freq_bottom_mhz + 0.0244140625 / 2.0, freq_top_mhz, 0.0244140625)

def rpl(freq, amp_pivot, spec_idx, running):
    PIVOT_FREQ = freq_bottom_mhz
    lfreq = np.log(freq / PIVOT_FREQ)
    lamp_fact = spec_idx * lfreq + running * lfreq ** 2
    return amp_pivot * np.exp(lamp_fact)


def bw_from_spec(amp_pivot, specidx, specrun):
    rpls = rpl(freq, amp_pivot, specidx, specrun)

    peak_frequency = freq[np.argmax(rpls)]

    pl_fwtm = np.max(rpls) / 10.
    idx = np.argwhere(np.diff(np.sign(rpls - pl_fwtm))).flatten()
    if len(idx) == 0:
        pl_fwtm_freq = (freq_bottom_mhz, freq_top_mhz)
    elif len(idx) == 2 and specrun > 0:
        if peak_frequency > 600:
            pl_fwtm_freq = (freq[idx[1]], freq_top_mhz)
        else:
            pl_fwtm_freq = (freq_bottom_mhz, freq[idx[0]])
    elif len(idx) == 2:
        pl_fwtm_freq = (freq[idx[0]], freq[idx[1]])
    elif rpls[0] < pl_fwtm:
        pl_fwtm_freq = (freq[idx[0]], freq_top_mhz)
    else:
        pl_fwtm_freq = (freq_bottom_mhz, freq[idx[0]])

    return pl_fwtm_freq[0], pl_fwtm_freq[1], peak_frequency, rpls

In [4]:
spec_index = []
spec_index_err = []
spec_run = []
spec_run_err = []
sp = data['spectral_index'].to_numpy()
sp_e = data['spectral_index_error'].to_numpy()
spr = data['spectral_running'].to_numpy()
spr_e = data['spectral_running_error'].to_numpy()
for i in range(len(sp)):
    if sp[i][0] == '[':
        sp[i] = sp[i][1:-1]
    x = np.array(ast.literal_eval(sp[i]))
    spec_index.append(x)
    if sp_e[i][0] == '[':
        sp_e[i] = sp_e[i][1:-1]
    y = np.array(ast.literal_eval(sp_e[i]))
    spec_index_err.append(y)
    if spr[i][0] == '[':
        spr[i] = spr[i][1:-1]
    x1 = np.array(ast.literal_eval(spr[i]))
    spec_run.append(x1)
    if spr_e[i][0] == '[':
        spr_e[i] = spr_e[i][1:-1]
    y1 = np.array(ast.literal_eval(spr_e[i]))
    print(x, y , x1, y1)
    spec_run_err.append(y1)
data['Spectral_Index'] = spec_index
data['Spectral_Running']= spec_run

1.901619786111957 1.610992635653052 -3.959472452646562 2.38115063616277
[11.75048601 59.71866727] [2.19487318 4.66958128] [ -55.98981776 -125.57288867] [8.43466425 9.62775671]
11.39992699213812 7.774890641099922 -7.062285731420766 7.084008669780858
-0.8061330616836221 2.598426162337671 -2.389732939868137 4.515962606192444
57.42647449050458 9.719755427596555 -109.15549103343841 18.27878693589127
7.2338814162032055 8.056960208539941 -136.13232803471593 66.67933465896508
27.05884903160525 8.22071158992608 -131.3564742663251 37.41377188542152
21.735708727071508 4.900103704355601 -60.85367673721211 13.276061216801958
-0.09156356017540901 2.5704288483414492 -1.146494757051617 3.833635157811746
0 0 0 0
0 0 0 0
0 0 0 0
0 0 0 0
0 0 0 0
0 0 0 0


In [5]:
amp = []
a = data['fitburst_amplitude'].to_numpy()
for i in range(len(a)):
    if a[i][0] == '[':
        a[i] = a[i][1:-1]
    x = np.array(ast.literal_eval(a[i]))
    amp.append(x)

data['amplitude'] = amp

In [6]:
mjds = []
toa = data['UTC_400'].to_numpy()
for i in range(len(toa)):
    x = ast.literal_eval(toa[i])
    print(x)
    # x = [n.strip() for n in x]
    # # z = parser.parse(x[0])
    # from astropy.time import Time
    try:
        if len(x) > 0:
            t = Time(x[0], format='iso', scale='utc')
            mjds.append(t.mjd)
        else:
            mjds.append(0)
    except:
        mjds.append(0)

['2020-10-23 07:48:30.782062']
['2020-11-23 05:52:02.538412', '2020-11-23 05:52:02.545423']
['2021-01-29 01:25:15.312581']
['2021-07-13 14:13:37.982290']
['2021-08-29 11:11:26.896288']
['2021-09-30 09:10:17.126028']
['2021-09-30 09:32:38.521022']
['2021-11-18 06:22:54.953842']
['2021-12-05 05:01:59.850211']
['2021-02-14 00:12:42.949611']
['2021-03-01 23:21:50.418838']
['2021-03-02 23:28:27.391257']
['2021-03-20 21:41:56.454417']
['2021-06-26 15:52:16.182347']
['2021-07-11 14:32:49.256215']


In [7]:
print(mjds)
data['MJD_400'] = mjds
ref_mjd = 58369.40
period = 16.33
data['phase'] = ((data['MJD_400']-ref_mjd)/period)%1

[59145.32535627387, 59176.24447382421, 59243.05920500673, 59408.59280072095, 59455.466283521855, 59487.38214266236, 59487.39766806739, 59536.26591381762, 59553.20972048855, 59259.00883043531, 59274.97350021803, 59275.97809480622, 59293.90412562983, 59391.66129840679, 59406.60612565064]


In [8]:
data['Downsample'] = int(384)

In [9]:
sc = []
sc_e = []
s = data['scattering_time_ms'].to_numpy()
print(s)
s_e = data['scattering_time_ms_error'].to_numpy()
print(s_e)
# for i in range(len(s)):
#     if s[i][0] == '[':
#         s[i] = s[i][1:-1]
#     x = ast.literal_eval(s[i])
#     sc.append(x)
#     if s_e[i][0] == '[':
#         s_e[i] = s_e[i][1:-1]
#     y = ast.literal_eval(s_e[i])
#     sc_e.append(y)
data['Scattering(s)'] = s.tolist()
data['Sc_err(s)'] = s_e.tolist()

[0.00528719 0.00094333 0.00928448 0.00315483 0.00446345 0.00411979
 0.00434866 0.00736993 0.002      0.00266201 0.00230256 0.00209175
 0.00208042 0.00217301 0.00056221]
[0.00000000e+00 1.08780635e-04 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 7.31045109e-06 6.17471441e-04 6.58436179e-04
 4.76775697e-04 8.75985880e-04 2.52846092e-06]


In [10]:
width = []
width_err = []
w = data['pulse_width_ms'].to_numpy()
w_e = data['pulse_width_ms_error'].to_numpy()
for i in range(len(w)):
    x = np.array(ast.literal_eval(w[i]))
    width.append(x)
    y = np.array(ast.literal_eval(w_e[i]))
    width_err.append(y)
data['Widths(s)'] = width
data['Width_err(s)']=width_err

print(width, width_err)

[array([0.0026436]), array([0.00083347, 0.00223258]), array([0.00464224]), array([0.00157742]), array([0.00223172]), array([0.0020599]), array([0.00217433]), array([0.00368497]), array([0.001]), array([0.00177903]), array([0.00183243]), array([0.00160314]), array([0.00057402]), array([0.00136403]), array([0.00036396])] [array([0.00029774]), array([0.00018786, 0.00019822]), array([0.0006023]), array([0.00030328]), array([0.00024443]), array([0.00023132]), array([0.00027823]), array([0.00037486]), array([0.00018556]), array([8.12356004e-06]), array([0.00033755]), array([0.00026012]), array([0.00014323]), array([0.00026766]), array([4.97827365e-06])]


In [11]:
s = data['Spectral_Index'].to_numpy()
r = data['Spectral_Running'].to_numpy()
a = data['amplitude'].to_numpy()

print(s)
print(r)
print(a)

[array(1.90161979) array([11.75048601, 59.71866727]) array(11.39992699)
 array(-0.80613306) array(57.42647449) array(7.23388142)
 array(27.05884903) array(21.73570873) array(-0.09156356) array(0)
 array(0) array(0) array(0) array(0) array(0)]
[array(-3.95947245) array([ -55.98981776, -125.57288867])
 array(-7.06228573) array(-2.38973294) array(-109.15549103)
 array(-136.13232803) array(-131.35647427) array(-60.85367674)
 array(-1.14649476) array(0) array(0) array(0) array(0) array(0) array(0)]
[array(5.26881008e-05) array([1.33275154e-04, 4.31662344e-07])
 array(1.41729635e-06) array(4.34804224e-05) array(8.35871246e-08)
 array(0.00013034) array(2.77192016e-05) array(2.05578801e-05)
 array(2.32956262e-05) array(-3.86962561) array(-3.91076191)
 array(-3.8825865) array(-3.61556289) array(-3.51134889)
 array(-4.15872468)]


In [12]:
bw = []
cf = []
for i in range(len(a)):
    if s[i].any() != 0:
        if a[i].size > 1:
            lo = np.zeros(len(a[i]))
            hi = np.zeros(len(a[i]))
            for j in range(len(a[i])):
                lo[j], hi[j], peak, pl = bw_from_spec(a[i][j],s[i][j],r[i][j])
                #print(lo[j], hi[j])
            print("Lowest and Highest is", np.min(lo), np.max(hi), peak)
            low = np.min(lo)
            high = np.max(hi)
        else:        
            low, high, peak, pl = bw_from_spec(a[i],s[i],r[i])
        print(low, high)
        b = high - low
        print("Bandwidth", b)
        c = (high+low)/2
        print("Central Frequency", c)
        print("\n")
        bw.append(b)
        cf.append(c)
    else:
        print("No data \n")
        bw.append(0.0)
        cf.append(0.0)
print(bw)
print(cf)

400.1953125 800.1953125
Bandwidth 400.0
Central Frequency 600.1953125


Lowest and Highest is 400.1953125 581.21337890625 507.62939453125
400.1953125 581.21337890625
Bandwidth 181.01806640625
Central Frequency 490.704345703125


501.06201171875 800.1953125
Bandwidth 299.13330078125
Central Frequency 650.628662109375


400.1953125 800.1953125
Bandwidth 400.0
Central Frequency 600.1953125


450.20751953125 601.98974609375
Bandwidth 151.7822265625
Central Frequency 526.0986328125


400.1953125 468.02978515625
Bandwidth 67.83447265625
Central Frequency 434.112548828125


400.1953125 506.40869140625
Bandwidth 106.21337890625
Central Frequency 453.302001953125


400.1953125 581.16455078125
Bandwidth 180.96923828125
Central Frequency 490.679931640625


400.1953125 800.1953125
Bandwidth 400.0
Central Frequency 600.1953125


No data 

No data 

No data 

No data 

No data 

No data 

[400.0, 181.01806640625, 299.13330078125, 400.0, 151.7822265625, 67.83447265625, 106.21337890625, 180.9692382812

In [13]:
band = data["Bandwidth"].to_numpy()
center = data["Central Frequency"].to_numpy()


bd = []
cnf = []

for i in range(len(band)):
    if band[i][0] == '[':
        band[i] = band[i][1:-1]
    x = ast.literal_eval(band[i])
    bd.append(x*2.575) ##multiplied by 2.575 to get 3 sigma 

    if center[i][0] == '[':
        center[i] = center[i][1:-1]
    y = ast.literal_eval(center[i])
    cnf.append(y)


print(band, center)
print(bd)
print(cnf)
#for i in range(len(bw)):

['0.0' '0.0' '0.0' '0.0' '0.0' '0.0' '0.0' '0.0' '0.0' '40.37254951976685'
 '51.90657993311264' '28.610219874167107' '11.508242108998923'
 '21.872062511916205' '26.19960077763819'] ['0.0' '0.0' '0.0' '0.0' '0.0' '0.0' '0.0' '0.0' '0.0' '668.0375455002746'
 '432.1598237245552' '489.9491616397749' '434.2402382421003'
 '396.3341520070669' '665.5066423310699']
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 103.95931501339965, 133.65944332776505, 73.67131617598031, 29.63372343067223, 56.32056096818423, 67.46397200241834]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 668.0375455002746, 432.1598237245552, 489.9491616397749, 434.2402382421003, 396.3341520070669, 665.5066423310699]


In [14]:
print(bw)
print(bd)
for i in range(len(bw)):
    if bw[i] > 0:
        bd[i] = bw[i]
        cnf[i] = cf[i]
print(cnf)
print(bd)
data["Bandwidth"] = bd
data["Central Frequency"] = cnf

[400.0, 181.01806640625, 299.13330078125, 400.0, 151.7822265625, 67.83447265625, 106.21337890625, 180.96923828125, 400.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 103.95931501339965, 133.65944332776505, 73.67131617598031, 29.63372343067223, 56.32056096818423, 67.46397200241834]
[600.1953125, 490.704345703125, 650.628662109375, 600.1953125, 526.0986328125, 434.112548828125, 453.302001953125, 490.679931640625, 600.1953125, 668.0375455002746, 432.1598237245552, 489.9491616397749, 434.2402382421003, 396.3341520070669, 665.5066423310699]
[400.0, 181.01806640625, 299.13330078125, 400.0, 151.7822265625, 67.83447265625, 106.21337890625, 180.96923828125, 400.0, 103.95931501339965, 133.65944332776505, 73.67131617598031, 29.63372343067223, 56.32056096818423, 67.46397200241834]


In [15]:
dm = []
d = data['fitburst_dm_error'].to_numpy()
print(d)
for i in range(len(d)):
    print(d[i])
    if d[i]>5:
        d[i]=0
    dm.append(d[i])
data['fitburst_d_error'] = dm
print(data['fitburst_d_error'])

[2.29113430e-02 3.48772000e+02 3.48772000e+02 3.15910920e-02
 2.27567614e-02 8.78587185e-02 5.51817392e-02 4.69073972e-02
 1.58562260e-02 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00]
0.02291134304379
348.772
348.772
0.031591091978303
0.022756761411079
0.087858718506713
0.055181739197075
0.046907397228231
0.015856226037773
0.0
0.0
0.0
0.0
0.0
0.0
0     0.022911
1     0.000000
2     0.000000
3     0.031591
4     0.022757
5     0.087859
6     0.055182
7     0.046907
8     0.015856
9     0.000000
10    0.000000
11    0.000000
12    0.000000
13    0.000000
14    0.000000
Name: fitburst_d_error, dtype: float64


In [16]:
# data_R3_intensity = data[['event_id','fitburst_dm','fitburst_dm_error','pulse_width_ms','pulse_width_ms_error',
#                          'scattering_time_ms','scattering_time_ms_error','Central Frequency','Bandwidth','MJD_400',
#                          'type_of_data','phase','Downsample']]
data_R3_intensity = data[['event_id','Downsample','Scattering(s)','Sc_err(s)',
                          'Widths(s)','Width_err(s)','Central Frequency','Bandwidth',
                          'MJD_400','phase','fitburst_dm','fitburst_d_error','type_of_data']]
data_R3_intensity.columns = ['Event_id','Downsample','Scattering(s)','Sc_err(s)','Widths(s)','Width_err(s)',
                            'Center_Freq','Bandwidth','TOA','phase','DM_fitburst','DM_err_fitburst', 'Type_of_data']
data_R3_intensity = data_R3_intensity.sort_values(by='Event_id')
#print(data_R3_intensity)
dm_i = data_R3_intensity['DM_fitburst'].to_numpy()
#print(dm_i.tolist())

In [17]:
data_R3_intensity.to_csv('R3_combine_intensity_pub.csv')

In [18]:
#### Edit and make it table worthy #####
sc_tab = []
dm_tab = []
w_tab = []

sc_err_tab = []
dm_err_tab = []

### All numpys ###

sc = data_R3_intensity["Scattering(s)"].to_numpy()
sc_err = data_R3_intensity["Sc_err(s)"].to_numpy()
dm = data_R3_intensity["DM_fitburst"].to_numpy()
dm_err = data_R3_intensity["DM_err_fitburst"].to_numpy()
bw = data_R3_intensity["Bandwidth"].to_numpy()
cf = data_R3_intensity["Center_Freq"].to_numpy()
width = data_R3_intensity['Widths(s)'].to_numpy()
width_err = data_R3_intensity['Width_err(s)'].to_numpy()

#### The width ####

w = np.zeros(len(width))
w_err = np.zeros(len(width))
w_ = []
w_err_ = []
down = data['Downsample'].to_numpy()
for i in range(len(w)):
#     x = width[i][1:-1].split()
#     y = width_err[i][1:-1].split()
#     x = [float(x) for x in x]
#     y = [float(y) for y in y]
    w_.append(width[i])
    w_err_.append(width_err[i])

for i in range(len(w_)):
    a = ''
    print("\n",i)
    for j in range(len(w_[i])):
#         print("This burst has {} components".format(len(w_[i])))
        if w_err_[i][j] < (2.56e-6):
            w_err_[i][j] = down[i]*2.56e-6
        if (0 < w_[i][j] < 0.1) and (w_[i][j] > 2.56e-6) and (w_[i][j] > w_err_[i][j]) :
            print("The width and error is", w_[i][j], w_err_[i][j])
            err = np.round(w_err_[i][j]*1e3,3)
            err = err*1e3
            err = str(int(err))
            if len(w_[i]) > 1:
                #a = a + str(+np.round(w_[i][j]*1e3,3))+'('+err+')'
                a = a + '{:.3f}'.format(round(w_[i][j]*1000, 3)) + '('+err+')'
            else:
                a = '{:.3f}'.format(round(w_[i][j]*1000, 3)) + '('+err+')'

        else:
            print("For burst {} the component {} is wrong".format(i+1, j+1))
            print("This width is wrong", w_[i][j], w_err_[i][j])
    w_tab.append(a)
    
print(w_tab)

##### The scattering #####

for i in range(len(sc_err)):
    x = (sc_err[i]*1e3).round(3)
    x = x*1e3
    x = str(int(x))

    sc_err_tab.append(x)
    
##### The DM #####
for i in range(len(dm_err)):
    if dm_err[i] < 1:
        x = (dm_err[i]).round(3)
        x = x*1e3
        x = str(int(x))
        dm_err_tab.append(x)
    else:
        dm_err_tab.append(str(dm_err[i].round(1)))
        
#### The bandwidth ####

freq_tab = []

for i in range(len(bw)):
    u = cf[i]+bw[i]/2
    l = cf[i]-bw[i]/2
    
    if u > 800:
        u = 800.0
    if l < 400:
        l = 400.0
    x = str(np.round(l,1))+'-'+str(np.round(u,1))
    freq_tab.append(x)
    

print(freq_tab)

for i in range(len(sc)):
    s = '{:.3f}'.format(round(sc[i]*1000, 3))+'('+sc_err_tab[i]+')'
    sc_tab.append(s)
    d_t = str(dm[i].round(3))+'('+dm_err_tab[i]+')'
    d_t = '{:.3f}'.format(round(dm[i], 3))+'('+dm_err_tab[i]+')'
    dm_tab.append(d_t)
print(sc_tab)
print("\n")
print(dm_tab)


 0
The width and error is 0.002643597325383 0.00029773530662000004

 1
The width and error is 0.0008334742227810001 0.00018786053053300003
The width and error is 0.0022325752734400003 0.000198223422507

 2
The width and error is 0.004642237502703 0.000602301481418

 3
The width and error is 0.001779030418484 8.123560044916047e-06

 4
The width and error is 0.001832425914638 0.00033755373790000004

 5
The width and error is 0.00160313585354 0.00026012247276800004

 6
The width and error is 0.00057402191802 0.00014323136656700002

 7
The width and error is 0.0013640287137640001 0.000267663624953

 8
The width and error is 0.000363959424903 4.978273649051842e-06

 9
The width and error is 0.001577417488698 0.000303284541738

 10
The width and error is 0.0022317226734050004 0.000244429814482

 11
The width and error is 0.0020598968150100003 0.00023132013280200003

 12
The width and error is 0.0021743316133050004 0.00027823387784400004

 13
The width and error is 0.0036849673246810003 0.00

In [19]:
data_table = data_R3_intensity[['Event_id','TOA','phase']]
b = []
for i in range(len(sc)):
    b.append('I'+str(i+1))
data_table['Burst'] = b
    
data_table["DM(pc/cc)"] = dm_tab
data_table["phase"] = data_table["phase"].round(3)
data_table["Scattering(ms)"] = sc_tab
data_table["Frequency Range(MHz)"] = freq_tab
data_table["MJD"] = data_table["TOA"].round(6)
data_table["Width(ms)"] = w_tab
#data_table["Width(ms)"] = (data_table["Widths(s)"]*1000).round(3)
data_table = data_table.reset_index(drop=True)
data_table=data_table[["Burst","Event_id","MJD","phase","DM(pc/cc)","Scattering(ms)","Width(ms)","Frequency Range(MHz)"]]
print(data_table)

   Burst   Event_id           MJD  phase    DM(pc/cc) Scattering(ms)  \
0     I1  139459007  59145.325356  0.515  350.401(23)       5.287(0)   
1     I2  144520399  59176.244474  0.409   348.772(0)     0.943(109)   
2     I3  154779432  59243.059205  0.500   348.772(0)       9.284(0)   
3     I4  157779605  59259.008830  0.477   348.772(0)       2.662(7)   
4     I5  162981697  59274.973500  0.455   348.772(0)     2.303(617)   
5     I6  163181404  59275.978095  0.516   348.772(0)     2.092(658)   
6     I7  165238765  59293.904126  0.614   348.772(0)     2.080(477)   
7     I8  175084048  59391.661298  0.600   348.772(0)     2.173(876)   
8     I9  175930797  59406.606126  0.515   348.772(0)       0.562(3)   
9    I10  176156139  59408.592801  0.637  348.722(32)       3.155(0)   
10   I11  183057477  59455.466284  0.507  349.294(23)       4.463(0)   
11   I12  188760049  59487.382143  0.462  349.912(88)       4.120(0)   
12   I13  188760957  59487.397668  0.463  349.530(55)       4.34

<ipython-input-19-5852244a7740>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_table['Burst'] = b
<ipython-input-19-5852244a7740>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_table["DM(pc/cc)"] = dm_tab
<ipython-input-19-5852244a7740>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#r

In [22]:
data_table.to_csv('Intensity_data_R3_table.csv')

In [103]:
bund = data_R3_intensity['Bandwidth'].to_numpy()
c = data_R3_intensity['Center_Freq'].to_numpy()
hi = np.zeros(len(bund))
lo = np.zeros(len(c))
for i in range(len(bund)):
    hi[i] = c[i]+bund[i]/2
    lo[i] = c[i]-bund[i]/2
print(hi.tolist())
print(lo.tolist())

[800.1953125, 581.21337890625, 800.1953125, 720.0172030069745, 498.98954538843776, 526.784819727765, 449.05709995743644, 424.49443249115905, 699.2386283322791, 800.1953125, 435.38818359375, 506.40869140625, 581.16455078125]
[400.1953125, 400.1953125, 501.06201171875, 616.0578879935748, 365.3301020606727, 453.11350355178473, 419.4233765267642, 368.17387152297476, 631.7746563298607, 400.1953125, 400.1953125, 400.1953125, 400.1953125]


In [ ]:
#### Next step is to split this csv file into events with data and events with no data. Especially in terms of UTC ###
### Then plot the scattering error one with a plus sign and the one with no scattering error with a triangle sign ####
#### Discuss more with Vicky and Daniele about this #####